<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020

<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>


# Validation and cross-validation

In this exercise you will implement a validation pipeline.

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.


In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2022-10-09 20:16:01--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2022-10-09 20:16:02--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc644f0a4db0d2f717b59c6484de.dl.dropboxusercontent.com/cd/0/get/BuhR7dlXzBxdxB-0n-INwiJTlE2itDME5qslt-MuUwmxIvf2NXLaAm2orX6bJFxoiSiR0HEDO1p0picRPEBI9vQPgS4NozhT75zp1Ngt3B-b0kVOFJGN_QJoS9wWd2Uvbom3-5Vx0xFKkP6qYMNbp31v-rIZRsRxeWl8pZ-44ZQflg/file?dl=1# [following]
--2022-10-09 20:16:02--  https://uc644f0a4db0d2f717b59c6484de.dl.dropboxusercontent.com/cd/0/get/BuhR7dlXzBxdxB-0n-INwiJTlE2itDME5qslt-MuUwmxIvf2NXLaAm2orX6bJFxoiSiR0

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn import preprocessing

np.random.seed(357)

In [2]:
def load(name: str) -> Tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != "cena"].to_numpy()
    y = data["cena"].to_numpy()

    return x, y

In [62]:
x_train, y_train = load("mieszkania.csv")
x_test, y_test = load("mieszkania_test.csv")

In [63]:
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(x_train[:, 1])
x_train[:, 1] = labelencoder.transform(x_train[:, 1])
x_test[:, 1] = labelencoder.transform(x_test[:, 1])

x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

In [72]:
#######################################################
# TODO: Implement the basic validation method,        #
# compare MSLE on training, validation, and test sets #
#######################################################
train_ratio = 0.8


def train_val_test_split(X: np.ndarray, y: np.ndarray):
    shuffle_ind = np.random.choice(X.shape[0], X.shape[0])

    shuffled_X = X[shuffle_ind]
    shuffled_y = y[shuffle_ind]

    split = int(X.shape[0] * train_ratio)

    return (
        shuffled_X[:split],
        shuffled_y[:split],
        shuffled_X[split:],
        shuffled_y[split:],
    )


x_tr, y_tr, x_val, y_val = train_val_test_split(x_train, y_train)

In [94]:
def predict(X, w):
    return X @ w


def mse_loss(y_hat, y):
    return ((y_hat - y) ** 2).mean()


def train(
    X_train,
    y_train,
    X_val,
    y_val,
    lr,
    n_epochs,
    predict_func=predict,
    loss_func=mse_loss,
):
    n = len(y_train)
    w = np.random.rand(X_train.shape[1])

    for _ in range(n_epochs):
        y_hat = predict_func(X_train, w)
        dw = 2 / n * X_train.T @ (y_hat - y_train)
        w -= lr * dw

        print()
        print(f"Train loss:\t {loss_func(y_hat, y_train)}")
        print(f"Validation loss: {loss_func(predict(X_val, w), y_val)}")

    return w

In [97]:
lr = 1e-7
n_epochs = 10000

w = train(x_tr, y_tr, x_val, y_val, lr, n_epochs)

print(f"Test loss:\t {mse_loss(predict(x_test, w), y_test)}")


Train loss:	 301289329628.0138
Validation loss: 90921834881.23087

Train loss:	 84407918755.98303
Validation loss: 78784115954.04883

Train loss:	 73099718299.82225
Validation loss: 77978987691.86899

Train loss:	 72478759953.09064
Validation loss: 77871157720.64595

Train loss:	 72413423058.31216
Validation loss: 77824288968.57225

Train loss:	 72376990684.05107
Validation loss: 77786246805.87086

Train loss:	 72342080159.53493
Validation loss: 77749969982.07031

Train loss:	 72307267918.94876
Validation loss: 77714097838.53691

Train loss:	 72272479943.63747
Validation loss: 77678332744.82027

Train loss:	 72237712374.44087
Validation loss: 77642607407.50848

Train loss:	 72202964999.61719
Validation loss: 77606906486.54417

Train loss:	 72168237797.05093
Validation loss: 77571226472.09856

Train loss:	 72133530754.49129
Validation loss: 77535566554.07909

Train loss:	 72098843860.20667
Validation loss: 77499926538.8003

Train loss:	 72064177102.49901
Validation loss: 77464306373.33

To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)


Let's now implement the method. Make sure that:

- number of partitions is a parameter,
- the method is not limited to `mieszkania.csv`,
- the method is not limited to one specific model.


In [103]:
####################################
# TODO: Implement cross-validation #
####################################

n_folds = 10


def train_n_folds():
    lr = 1e-7
    n_epochs = 1000

    losses = []

    for train_idx, val_idx in get_fold_indices(x_train, n_folds):
        x_tr_fold = x_train[train_idx]
        y_tr_fold = y_train[train_idx]

        x_val_fold = x_train[val_idx]
        y_val_fold = y_train[val_idx]

        w = train(
            x_tr_fold, y_tr_fold, x_val_fold, y_val_fold, lr, n_epochs
        )  # it doesn't make much sense here as we reset w each fold although the n-fold cross-validation seems to be working fine

        losses.append(mse_loss(predict(x_val_fold, w), y_val_fold))

    print(f"Validation losses: {losses}")
    print(f"Average validation loss: {np.mean(losses)}")


def get_fold_indices(X: np.ndarray, n_folds: int):
    fold_size = X.shape[0] // n_folds
    val_start_index = 0
    folds = []
    X_indices = np.arange(X.shape[0])

    for _ in range(n_folds):
        train_indices = np.concatenate(
            [X_indices[:val_start_index], X_indices[val_start_index + fold_size :]]
        )
        val_indices = X_indices[val_start_index : val_start_index + fold_size]

        folds.append((train_indices, val_indices))
        val_start_index += fold_size

    return folds


train_n_folds()


Train loss:	 310949776660.81946
Validation loss: 153041355270.35626

Train loss:	 82035576040.84366
Validation loss: 113848116813.53755

Train loss:	 70070520230.851
Validation loss: 106953175932.80898

Train loss:	 69416807154.58844
Validation loss: 105454250525.09988

Train loss:	 69352863636.33754
Validation loss: 105084122928.30626

Train loss:	 69319686749.00668
Validation loss: 104966336927.13394

Train loss:	 69288131108.11049
Validation loss: 104905896962.06845

Train loss:	 69256677117.5855
Validation loss: 104858552077.01126

Train loss:	 69225245538.68579
Validation loss: 104814213817.76068

Train loss:	 69193832230.22714
Validation loss: 104770579747.54414

Train loss:	 69162436966.47841
Validation loss: 104727124258.12405

Train loss:	 69131059725.85854
Validation loss: 104683727347.61868

Train loss:	 69099700497.4361
Validation loss: 104640361610.41708

Train loss:	 69068359270.84052
Validation loss: 104597020780.13393

Train loss:	 69037036035.73616
Validation loss: 10

Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.


In [105]:
##############################
# TODO: Investigate the data #
##############################

pd.DataFrame(x_train).describe()
pd.DataFrame(y_train).describe()

,0
count,2.000000e+02
mean,5.079195e+05
std,2.723625e+05
min,1.025720e+05
25%,3.051080e+05
50%,4.556925e+05
75%,6.628898e+05
max,1.102309e+06
